# Steam Review Sentiment

A machine learning project to predict Steam game ratings (thumbs up/down) using review text and game metadata.

# Data Overview TODO

- Our data is using 700 rows with 5 features to determine a Steam game rating.
- The data was collected through a script we ran on our local machine to pull data from the `/review` endpoint from the Steam API.
- When collecting this data, we did not need to perform any initial transformations
- TODO: {Provide line about cleaning or transforming data during the feature engineering}
- TODO: {Add if there were any additional exploration performed when feature engineering}
- TODO: {Answer the following: What is your dependent variable? regression or classification? distribution?}

### Data Descriptions

#### Continuous
- VotesUp: the number of users that found this review helpful
- VotesFunny: the number of users that found this review funny
- PlaytimeTotal: lifetime playtime tracked in this app
- PlaytimeReview: playtime when the review was written
- PlaytimeTwoWeek: playtime tracked in the past two weeks for this app
- NumberofReviews: number of reviews written by the user
- PostedDate: date the review was created (unix timestamp)

#### Categorical
- AppID:
- GameName:
- ReviewID:
- Author:
- Review:
- Recommended: true means it was a positive recommendation

In [5]:
import pandas as pd

# Load the dataset
file_path = "Dataset/steamreviews.csv"
steam_reviews = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(steam_reviews.head())

     AppID                  GameName   ReviewID             Author  \
0  3409570            Bricks Smasher  189034666  76561199483805980   
1  3409700  Honekawa Marionette Demo  186347944  76561198991875232   
2  3409750          Card Draw (Demo)  192050113  76561198973629547   
3  3409750          Card Draw (Demo)  191604297  76561198941797474   
4  3409750          Card Draw (Demo)  189258437  76561198261537547   

                                              Review  Recommended  VotesUp  \
0  Good, clean, silly fun! [i]Bricks Smasher[/i] ...         True        1   
1  I loved the art style and music. I love the lo...         True        1   
2  Very interesting game. Strategic mode is my fa...         True        1   
3  (I've been playing this on itch.io for a while...         True        1   
4  Very good game the best mode is strategy I sug...         True        1   

   VotesFunny  PlaytimeTotal  PlaytimeReview  PlaytimeTwoWeeks  \
0           0             56            48.0

## Install and Import PyTorch

Install PyTorch if not already installed and import the necessary modules.

In [1]:
# Install PyTorch (uncomment the line below if PyTorch is not installed)
# !pip install torch

# Import PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

# Tensors in PyTorch
Create and manipulate tensors, including initialization, reshaping, and basic properties.

In [2]:
# Create a tensor
tensor = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)

# Display tensor properties
print("Tensor:\n", tensor)
print("Shape:", tensor.shape)
print("Data type:", tensor.dtype)

# Reshape the tensor
reshaped_tensor = tensor.view(4, 1)
print("Reshaped Tensor:\n", reshaped_tensor)

Tensor:
 tensor([[1., 2.],
        [3., 4.]])
Shape: torch.Size([2, 2])
Data type: torch.float32
Reshaped Tensor:
 tensor([[1.],
        [2.],
        [3.],
        [4.]])


## Training TODO

10 pts: perform some thoughtful supervised learning, including engineering and selecting features, selecting
and optimizing a model, and explaining your model (coeﬃcients or feature importance, performance). Here
are some suggested key points.
- feature engineering / selection, bivariate charts? Interactions?
- missing data? how to handle it?
- Selection of modeling algorithm? classification or regression? binary or multi-class?
- interpretation of variable importance, coeﬃcients if applicable
- justification of choice of metric (accuracy, precision / recall, other?)
- is class weighting or over / under sampling appropriate?
- discussion of choice or tuning of hyperparameters, if any
- meaningful discussion of predictive power and conclusions from model
- look at misclassified examples from test dataset, what do they say about your model?
- outliers in data?

## PCA TODO

5 pts: PCA as data exploration and visualization. Here are some suggested key points.
- take a look at PCA, percent explained
- take a look at top eigenvector or two, what is it made out of?
- can you visualize your prediction problem by projecting to 2 dimensions?

## K-means and data exploration TODO

5 pts: k-means as data exploration and visualization. Here are some suggested key points.
- discussion for choosing number of clusters
- analysis of cluster centers
- scatter plot(s) showing 2 dimensional perspective of clusters and cluster centers?
- meaningful interpretation / discussion of conclusions